In [72]:
import deepdish as dd
import os
import pandas as pd
import sqlite3
%run 'matrix_factorization_system.ipynb'

Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [30]:
HOME_DIR = '/home/jvidyala/final_data/'
TRAIN_DATA = HOME_DIR + 'mf_train_data/'
TEST_DATA = HOME_DIR + 'mf_test_data/'
WEIGHTS_DIR = HOME_DIR + 'model_weights/'

In [17]:
#Song and user id to categorical var mapping
#Stored as {actual_id:categorical_id} in normal, reverse has opposite
def mappings():
    if 'taste_profile.h5' not in os.listdir(HOME_DIR):
        dataset = pd.read_csv("/home/jvidyala/data/train_triplets.txt",sep="\t",skiprows=1,names="user_id,song_id,play_count".split(","))
        dd.io.save(HOME_DIR + 'taste_profile.h5',dataset)
    else:
        dataset = dd.io.load(HOME_DIR + 'taste_profile.h5')
        
    df = dataset.copy(deep=True)

    dataset.user_id = dataset.user_id.astype('category')
    dataset.song_id = dataset.song_id.astype('category')

    dataset.user_id = dataset.user_id.cat.codes.values
    dataset.song_id = dataset.song_id.cat.codes.values
    song_id_mappings = {}
    song_id_mappings_reverse = {}
    user_id_mappings = {}
    user_id_mappings_reverse = {}

    for song_id,cat_id in zip(df.song_id.values,dataset.song_id.values):
        song_id_mappings[song_id] = cat_id
        song_id_mappings_reverse[cat_id] = song_id

    for user_id,cat_id in zip(df.user_id.values,dataset.user_id.values):
        user_id_mappings[user_id] = cat_id
        user_id_mappings_reverse[cat_id] = user_id


    dd.io.save(HOME_DIR + 'song_id_mappings.h5', song_id_mappings)
    dd.io.save(HOME_DIR + 'song_id_mappings_reverse.h5', song_id_mappings_reverse)
    dd.io.save(HOME_DIR + 'song_id_mappings.h5', song_id_mappings)
    dd.io.save(HOME_DIR + 'user_id_mappings_reverse.h5', user_id_mappings_reverse)

In [61]:
#Returns song name from categorical variable listening history 
def song_name_lookup(listening_history):
    song_id_mappings_reverse = dd.io.load(HOME_DIR + 'song_id_mappings_reverse.h5')
    user_history = [song_id_mappings_reverse[cat_id] for cat_id in listening_history]
    user_history_song_names = []
    
    conn = sqlite3.connect(HOME_DIR+'track_metadata.db')
    for song_id in user_history:
        query = "SELECT DISTINCT title FROM songs WHERE song_id='%s'" % song_id
        cursor = conn.execute(query)
        for row in cursor:
            user_history_song_names.append(row)
    return user_history_song_names

In [67]:
#Create test playlist for given user id, stored one has uid=732659 
def create_test_playlist(uid):
    a_songid = dd.io.load('model_sample_data/a_songid.h5')
    a_userid = dd.io.load('model_sample_data/a_userid.h5')
    
    listening_history = []
    for user_id,song_id in zip(a_userid,a_songid):
        if user_id == uid:
            listening_history.append(song_id)
        else: break
    listening_history = [int(x) for x in listening_history]
    
    user_history = song_name_lookup(listening_history)
    dd.io.save(HOME_DIR + 'test_playlist_user_732659.h5',user_history)
    dd.io.save(HOME_DIR + 'test_playlist_user_732659_cat.h5', listening_history)

Functions for testing generated playlist. Create new ones for on-the-fly testing

In [74]:
global history_stack 
history_stack = []

In [75]:
def get_mf_predictions(song,uid=732659):
    user_predictions,prediction_list = main(uid)
    
    t = prediction_list
    history_stack.append(song)
    for elem in history_stack:
        t.remove(elem)
    
    return t

def playlist_test():
    playlist = dd.io.load(HOME_DIR + 'test_playlist_user_732659_cat.h5')
    playlist[3] = 49524
    playlist[2] = 244554
    playlist[1] = 248348
    history_stack = []
    for song in playlist[:20]:
        deep_nmf_pred = get_mf_predictions(song)
        

In [68]:
mappings()
create_test_playlist(732659)

In [78]:
playlist_test()

hello


In [3]:
dataset = dd.io.load(HOME_DIR + 'taste_profile.h5')

In [8]:
dataset.loc[dataset['user_id']==732659]

,user_id,song_id,play_count


In [65]:
a_songid = dd.io.load(HOME_DIR + 'song_id_mappings.h5').values()
